# Pre trained network
![imagen](https://www.researchgate.net/publication/336874848/figure/fig1/AS:819325225144320@1572353764073/Illustrations-of-transfer-learning-a-neural-network-is-pretrained-on-ImageNet-and.png)

Estas son las arquitecturas de redes neuronales más utilizadas en la comunidad. Para más detalle sobre el funcionamiento de cada red, consultar el *Hands on Machine Learning for Python*.
* VGG-16
* VGG-19
* Inception V3
* XCeption
* ResNet-50

Estas redes las podemos incorporar entrenadas, o sin entrenar.

## ResNet50V2

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import cv2

from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

In [2]:
# image-net.org
base_model = ResNet50V2(input_shape=(224, 224, 3),
                       include_top=True,
                       weights='imagenet',
                       classifier_activation='softmax')

In [3]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

Cargamos algunas imagenes desde local, para ver qué tal funciona la red ResNet50V2 ya entrenada.

In [4]:
import os


def read_data(path):
    X = []

    for file in os.listdir(path):
        image = imread(path + '/' + file)
        smallimage = cv2.resize(image, (224, 224))
        print(path + '/' + file)
        
        X.append(smallimage)

    return np.array(X)
    

x_test = read_data('img')

# Procesar las imagenes tal y como entran en el modelo
x_test = preprocess_input(x_test)
print(x_test.shape)

img/bear-1.jpg
img/cat.8016.jpg
img/cat.8037.jpg
img/dog.11856.jpg
img/dog.11857.jpg
img/horse.jpg
img/karate.jpg
img/pizza.jpg
(8, 224, 224, 3)


In [5]:
preds = base_model.predict(x_test)

decodes = decode_predictions(preds, top=5)

for i in decodes:
    print('############')
    
    for j, decode in enumerate(i):
        print('Predicted:\n', decode[1], decode[2])

############
Predicted:
 brown_bear 0.99944514
Predicted:
 chow 0.00054139504
Predicted:
 American_black_bear 8.784533e-06
Predicted:
 ice_bear 1.1080934e-06
Predicted:
 howler_monkey 4.4057498e-07
############
Predicted:
 Egyptian_cat 0.659723
Predicted:
 Siamese_cat 0.15741089
Predicted:
 tiger_cat 0.06260011
Predicted:
 lynx 0.022836583
Predicted:
 tabby 0.018356169
############
Predicted:
 Egyptian_cat 0.8211865
Predicted:
 lynx 0.12932149
Predicted:
 Siamese_cat 0.021827206
Predicted:
 tabby 0.018324552
Predicted:
 tiger_cat 0.006797261
############
Predicted:
 Rottweiler 0.76189536
Predicted:
 Brabancon_griffon 0.0746136
Predicted:
 Staffordshire_bullterrier 0.028572641
Predicted:
 EntleBucher 0.019904258
Predicted:
 Doberman 0.019285118
############
Predicted:
 collie 0.99525565
Predicted:
 Shetland_sheepdog 0.004744308
Predicted:
 yellow_lady's_slipper 9.751216e-09
Predicted:
 goldfinch 8.175372e-09
Predicted:
 groenendael 7.475547e-09
############
Predicted:
 standard_poodle 0

## VGG16
En este caso vamos a importar la red VGG16, que utilizaremos como red preentrenada y completaremos con una fully connected layer. 

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

IM_SIZE=64

TRAIN_PATH = 'C:/Users/Daney/Desktop/dogs&cats/mini_train/train/'
filenames = os.listdir(TRAIN_PATH)
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    categories.append(category)
    
df = pd.DataFrame({
    'filenames': filenames,
    'category': categories
})

train_df, validate_df = train_test_split(df,
                                         test_size=0.20,
                                         random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [8]:
df.head()

,filenames,category
0,cat.0.jpg,cat
1,cat.1.jpg,cat
2,cat.10.jpg,cat
3,cat.100.jpg,cat
4,cat.1000.jpg,cat


In [11]:
from keras.preprocessing.image import ImageDataGenerator


# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale = 1.0/255. )

In [12]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 5260 validated image filenames belonging to 2 classes.
Found 1316 validated image filenames belonging to 2 classes.


In [13]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (IM_SIZE, IM_SIZE, 3),
                  include_top=False,
                  weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False
    
##### FULLY CONNECTED LAYER #####
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [15]:
vgghist = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 5)

Epoch 1/5
263/263 [==============================] - 230s 875ms/step - loss: 0.5462 - accuracy: 0.7247 - val_loss: 0.4585 - val_accuracy: 0.7774
Epoch 2/5
263/263 [==============================] - 192s 729ms/step - loss: 0.5174 - accuracy: 0.7451 - val_loss: 0.4547 - val_accuracy: 0.7910
Epoch 3/5
263/263 [==============================] - 192s 730ms/step - loss: 0.5100 - accuracy: 0.7511 - val_loss: 0.4647 - val_accuracy: 0.7781
Epoch 4/5
263/263 [==============================] - 189s 719ms/step - loss: 0.5043 - accuracy: 0.7529 - val_loss: 0.4348 - val_accuracy: 0.8024
Epoch 5/5
263/263 [==============================] - 202s 769ms/step - loss: 0.5006 - accuracy: 0.7576 - val_loss: 0.4624 - val_accuracy: 0.7705


### Inception

In [ ]:
# TODO

## ResNet50V2 sin entrenar

In [16]:
IM_SIZE=32

# Entrono todo de 0
base_model = ResNet50V2(input_shape=(IM_SIZE, IM_SIZE, 3),
                        include_top=False)

# Entreno todo, partiendo de la convolucional entrenada
base_model = ResNet50V2(input_shape=(IM_SIZE, IM_SIZE, 3),
                        include_top=False,
                        weights='imagenet')

In [17]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 5260 validated image filenames belonging to 2 classes.
Found 1316 validated image filenames belonging to 2 classes.


In [18]:
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [19]:
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 5)

Epoch 1/5
 28/263 [==>...........................] - ETA: 10:31 - loss: 0.9910 - accuracy: 0.5536

KeyboardInterrupt: 